# Dataset Preparation

## Libraries

Teresa
For the dataset preparation, we will be doing: web scrapping from several sites, requesting API access and using Azure databases. For that, we will require several libraries stated bellow:
1. Web Scrappping: bs4, requests, re and urllib
2. API requests: requests
3. Azure access: pyobdc

Nuno
For the dataset preparation, since we will be doing web scrapping from several sites, requesting api access and also using Azure databases we will require several libraries stated bellow:

In [1]:
#Libraries for web scrapping
#For further information
#Beautiful Soup: https://beautiful-soup-4.readthedocs.io/en/latest/
#https://docs.python-requests.org/en/latest/
#https://docs.python.org/3/library/re.html
#https://docs.python.org/3/library/urllib.request.html
from bs4 import BeautifulSoup
import requests
import re
import urllib.request

#Module to work and make access Azure
#For further Documentation visit: https://pypi.org/project/pyodbc/
import pyodbc

#NumPy and Pandas libraries
import numpy as np
import pandas as pd

In [2]:
#Additional settings, from Pandas, to display all columns and 20 rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

1. Web Scrappping: bs4, requests, re and  urllib
2. API requests: requests
3. Azure access: pyobdc

Teresa:

To gather information from the web, we will require to repeat the same code several times, to extract the same information for each year (2015 to 2021). We defined four functions, because of the changes occured in the websites structures, in order to have more control over the information that we will be extracting.

The four functions are:
1. Links_Extraction
2. fia_f1_data
3. fia_f1_session
4. f1_gp_circuits

Nuno
## Functions

To gather information from the web we will require to repeat the same code several times, meaning that we will extract the same information for each year (2015 to 2021). The reason why we haven't coded only one function is because the website structures do change all over the years and this way we will have more control over the information we will be extracting.

We have prepared 4 functions to help extracting the information each year:
1. Links_Extraction
2. fia_f1_data
3. fia_f1_session
4. f1_gp_circuits

Let's cover a brief description on what they do.

### Links_Extraction

This function is used to extract all the relevant links to pass to another function for extraction. The output is a list.

We have 2 paramenters **url** which is the page we want to know the links available, lets say https://www.fia.com/f1-archives?season=1108 and since these pages have a lot of links we are only interested in a few, we added another parameter called **url_string**, where we pass a string contained in the url that we want. In case we want the main information from the race we know that those links contain the string *'race-classification'* so we would search only for those.

The function is defined below.


In [3]:

def Links_Extraction(url, url_string):
    '''
    Description: Function to extract links from a given url and store it to a list
    Parameters:
        url: Webpage to search for any given links
        url_string: Search for links that contain a specific string
    Usage: 
        Links_Extraction(url = 'https://www.fia.com/f1-archives', url_string = 'session')
    '''
    
    response = requests.get(url)
    html_document = response.text
    soup = BeautifulSoup(html_document, 'html.parser')
    
    links = []
    for link in soup.find_all('a', attrs={'href': re.compile(url_string)}):
        links.append(link.get('href'))
    
    return links

So let us try this code with the link provided above and for the *race classification* links:

In [4]:
Links_Extraction(url = 'https://www.fia.com/f1-archives?season=1108', url_string = 'race-classification')

['/events/fia-formula-one-world-championship/season-2021/bahrain-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/emilia-romagna-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/portuguese-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/spanish-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/monaco-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/azerbaijan-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/french-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/styrian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/austrian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/british-grand-prix/race-classification',
 '/event

If needed, we can tweek the output to just the relevant pages, meaning that during the COVID outbreak there were some cancelations and some grand prix pages were setup, but no data was added since they were cancelled. This function can the be applied to any page.

### f1_gp_circuits

This function extracts the name of the race and date of the actual Grand Prix (GP) from the espn.com website - https://www.espn.com/f1/schedule/_/year/2022. With this information, we will use it to check the links we will feed on the other functions.

This function will do a small prerocessing on the race field in order to prepare a colummn called GP with a simple string to identify the Grand Prix, which will be used to merge more data at a later time.

This function only uses one parameter - season and by default is 2022

In [5]:
# grand prix country list taken by date of ocorrence
def f1_gp_circuits(season = 2022):
    '''
    Description: Extracts the Race Name and GP and ordered by date of occurence from espn.com
    Parameters:
        season: Year to which we want to retrieve information on the Grand Prix. By default season is set to 2022
    Usage:
        f1_gp_circuits(season = 2021)
    '''
    print('Season: ' + str(season) + ' | Source: espn.com')
    url = f'https://www.espn.com/f1/schedule/_/year/{season}'

    source = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(source,'lxml')
    table = soup.find_all('table')[0]
    df = pd.read_html(str(table), flavor='bs4', header=[0])[0]
    df.drop(['Winner/Lights Out','TV'],axis=1, inplace=True)

    gp = df['Race'].str.split(' GP',n = 1, expand = True)
    df['GP'] = gp[0].str.lower()
    # correction on the names that will feed the links
    df['GP'] = df['GP'].str.replace('socar ', '')
    df['GP'] = df['GP'].str.replace('vtb ', '')
    df['GP'] = df['GP'].str.replace('arabian', 'arabia')
    df['GP'] = df['GP'].str.replace('etihad airways ', '')
    df['GP'] = df['GP'].str.replace(' ', '-')
    df['GP'] = df['GP'].str.replace('singapore-airlines-singapore-gmarina-bay-street-circuit', 'singapore')
    df['GP'] = df['GP'].str.replace('mercedes-benz-german', 'german')
    df['GP'] = df['GP'].str.replace('rolex-british','british')
    return df

We can use this function by simply choosing a season.

In [7]:
f1_gp_circuits(season = 2019)

Season: 2019 | Source: espn.com


,Date,Race,GP
0,Mar 15 - 17,Australian GPMelbourne Grand Prix Circuit,australian
1,Mar 29 - 31,Bahrain GPBahrain International Circuit,bahrain
2,Apr 12 - 14,Chinese GPShanghai International Circuit,chinese
3,Apr 26 - 28,Socar Azerbaijan GPBaku City Circuit,azerbaijan
4,May 10 - 12,Spanish GPCircuit de Barcelona-Catalunya,spanish
...,...,...,...
16,Oct 11 - 13,Japanese GPSuzuka International Racing Course,japanese
17,Oct 25 - 27,Mexican GPAutodromo Hermanos Rodriguez,mexican
18,Nov 1 - 3,United States GPCircuit of the Americas,united-states
19,Nov 15 - 17,Brazilian GPAutodromo Jose Carlos Pace,brazilian


### fia_f1 data and season functions

Although, these are two different functions they work on the same principle, we feed them with the **season**, **gp_city** and the **gp_links** list we have prepared previously with the function Links_Extraction.

#### fia_f1_data

This function will extract data from the race classification page, regarding team, pilots, classification fastest laps, best sector times, speed traps, maximum speeds and pit stops for each GP.

#### fia_f1_season

This function will extract data from the season classification page, regarding initial qualification and grid for each GP.

Both functions use BeautifulSoup package to extract information from the tables the page holds. That information is selected and renamed appropriately for each GP link we feed. In the end we have a data set that holds all this information for all the GP contained in that season.

Check the code below for the two functions:

In [8]:

def fia_f1_data(season, gp_city, gp_link):
    '''
    Description: Prepare dataset by season for team, pilots, classification fastest laps, best sector times,
    speed traps, maximum speeds and pit stops for each GP
    Parmeters:
        season: corresponding season (year)
        gp_city: list generated by f1_gp_circuits function
        gp_links: list of links corresponding to different races for the same season
    Usage:
        df_2021_data = fia_f1_session(2021, gp_city = city_21, gp_link = links_data_21)
    '''
    print('Season: ' + str(season) + ' | Source: fia.com')

    dataset = pd.DataFrame()
    n = 0
    for i in gp_link:
        url = f'https://www.fia.com{i}'
        print('Circuit: ' + str(gp_city[n]) + ' | ' + url)
        
        source = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(source,'lxml')

        # classification
        table_class = soup.find_all('table')[0] 
        df_class = pd.read_html(str(table_class), flavor='bs4', header=[1])[0]
        df_class.drop(['Unnamed: 5'],axis=1, inplace=True)
        name = df_class['DRIVER'].str.split(' ',n = 1, expand = True)
        df_class['DRIVER_SNAME'] = name[0].astype(str).str[0] + '. ' + name[1]
        df_class['GRAND_PRIX'] = gp_city[n].capitalize()
        df_class['SEASON'] = season
        df_class = df_class.rename (columns={'POS': 'CLASS', 'TIME': 'RACE_TIME'}) 
        n = n + 1

        # fastest laps
        table_flaps = soup.find_all('table')[1] 
        df_flaps = pd.read_html(str(table_flaps), flavor='bs4', header=[1])[0]
        df_flaps.drop(['Unnamed: 7','Unnamed: 8'],axis=1, inplace=True)
        df_flaps = df_flaps.rename(columns={'POS': 'FLAP_POS', 'LAP TIME': 'FLAP_TIME', 'LAP': 'F_LAP', 'GAP': 'FLAP_GAP', 'KM/H': 'FLAP_KM/H', 'TIME': 'FLAP_HOUR'}) 

        # best sector times
        table_bs = soup.find_all('table')[2] 
        df_bs = pd.read_html(str(table_bs), flavor='bs4', header=[2])[0]
        df_bs.drop(['Unnamed: 7'],axis=1, inplace=True)
        df_bs_1 = df_bs[['POS', 'DRIVER',   'TIME']]
        df_bs_1 = df_bs_1.rename(columns={'POS': 'BS1_POS', 'DRIVER':   'BS1_DRIVER', 'TIME':   'BS1_TIME'}) 
        df_bs_2 = df_bs[['POS', 'DRIVER.1', 'TIME.1']]
        df_bs_2 = df_bs_2.rename(columns={'POS': 'BS2_POS', 'DRIVER.1': 'BS2_DRIVER', 'TIME.1': 'BS2_TIME'}) 
        df_bs_3 = df_bs[['POS', 'DRIVER.2', 'TIME.2']]
        df_bs_3 = df_bs_3.rename(columns={'POS': 'BS3_POS', 'DRIVER.2': 'BS3_DRIVER', 'TIME.2': 'BS3_TIME'}) 

        # speed traps
        table_straps = soup.find_all('table')[3] 
        df_straps = pd.read_html(str(table_straps), flavor='bs4', header=[1])[0]
        df_straps.drop(['TEAM','TIME','Unnamed: 5'],axis=1, inplace=True)
        df_straps = df_straps.rename(columns={'POS': 'ST_POS', 'KM/H': 'ST_KM/H'})

        # maximum speeds
        table_mspeeds = soup.find_all('table')[4] 
        df_mspeeds = pd.read_html(str(table_mspeeds), flavor='bs4', header=[2])[0]
        df_mspeeds.drop(['Unnamed: 7'],axis=1, inplace=True)
        df_mspeeds_1 = df_mspeeds[['POS', 'DRIVER',   'KM/H']]
        df_mspeeds_1 = df_mspeeds_1.rename(columns={'POS': 'I1_POS', 'DRIVER':   'I1_DRIVER', 'KM/H':   'I1_KM/H'}) 
        df_mspeeds_2 = df_mspeeds[['POS', 'DRIVER.1', 'KM/H.1']]
        df_mspeeds_2 = df_mspeeds_2.rename(columns={'POS': 'I2_POS', 'DRIVER.1': 'I2_DRIVER', 'KM/H.1': 'I2_KM/H'}) 
        df_mspeeds_3 = df_mspeeds[['POS', 'DRIVER.2', 'KM/H.2']]
        df_mspeeds_3 = df_mspeeds_3.rename(columns={'POS': 'FL_POS', 'DRIVER.2': 'FL_DRIVER', 'KM/H.2': 'FL_KM/H'}) 

        # pit stops
        table_pstops = soup.find_all('table')[5] 
        df_pstops = pd.read_html(str(table_pstops), flavor='bs4', header=[1])[0]
        df_pstops = df_pstops.rename(columns={'NO': 'DRIVER_NO', 'TOTAL TIME':   'PS_TOTAL_TIME'})
        t = df_pstops.index[df_pstops['DRIVER_NO'] == 'RACE - PIT STOP - DETAIL'].to_list() 
        df_pstops = df_pstops[:t[0]]
        df_pstops = df_pstops[['DRIVER_NO','DRIVER','STOPS','PS_TOTAL_TIME']]

        # merge information for maximum speeds and best sector tables 
        fia_df = pd.merge(df_class, df_flaps, how='left', on=['DRIVER'])

        fia_df = pd.merge(fia_df, df_bs_1,  how='left', left_on='DRIVER_SNAME', right_on='BS1_DRIVER')
        fia_df = pd.merge(fia_df, df_bs_2,  how='left', left_on='DRIVER_SNAME', right_on='BS2_DRIVER')
        fia_df = pd.merge(fia_df, df_bs_3,  how='left', left_on='DRIVER_SNAME', right_on='BS3_DRIVER')
        fia_df.drop(['BS1_DRIVER','BS2_DRIVER','BS3_DRIVER'],axis=1, inplace=True)

        fia_df = pd.merge(fia_df,   df_straps, how='left', on=['DRIVER'])

        fia_df = pd.merge(fia_df, df_mspeeds_1,  how='left', left_on='DRIVER_SNAME', right_on='I1_DRIVER')
        fia_df = pd.merge(fia_df, df_mspeeds_2,  how='left', left_on='DRIVER_SNAME', right_on='I2_DRIVER')
        fia_df = pd.merge(fia_df, df_mspeeds_3,  how='left', left_on='DRIVER_SNAME', right_on='FL_DRIVER')
        fia_df.drop(['I1_DRIVER','I2_DRIVER','FL_DRIVER'],axis=1, inplace=True)

        fia_df = pd.merge(fia_df, df_pstops, how='left', on=['DRIVER'])

        dataset = dataset.append(fia_df)
        
    return dataset

In [ ]:
# function to prepare dataset by season - extracts qualification and grid position
def fia_f1_session(season, gp_city, gp_link):
    '''Insert the season = year and grand_prix as a list with fia links'''
    print('Season: ' + str(season) + ' | Source: fia.com')

    dataset = pd.DataFrame()
    n = 0
    for i in gp_link:
        url = f'https://www.fia.com{i}'
        print('Circuit: ' + str(gp_city[n]) + ' | ' + url)
        
        source = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(source,'lxml')    
        
        #qualification
        table_qual = soup.find_all('table')[3] 
        df_qual = pd.read_html(str(table_qual), flavor='bs4', header=[1])[0]
        df_qual = df_qual[['POS','DRIVER','Q1','LAPS','Q2','LAPS.1','Q3','LAPS.2']]
        df_qual = df_qual.rename (columns={'POS': 'QL_CLASS', 'Q1': 'QL_TIME1','Q2': 'QL_TIME2','Q3': 'QL_TIME3',
                                           'LAPS':'QL_LAPS1','LAPS.1':'QL_LAPS2','LAPS.2':'QL_LAPS3'})
        df_qual['GRAND_PRIX'] = gp_city[n].capitalize()
        df_qual['SEASON'] = season

        #grid
        table_grid = soup.find_all('table')[4] 
        df_grid = pd.read_html(str(table_grid), flavor='bs4', header=[1])[0]
        df_grid = df_grid[['POS','DRIVER','TIME']]
        df_grid = df_grid.rename (columns={'POS': 'GD_CLASS', 'TIME': 'GD_TIME'})

        fia_df = pd.merge(df_qual, df_grid, how='left', on=['DRIVER'])
        n = n + 1
        dataset = dataset.append(fia_df)

    return dataset

Please see the usage example on the race classification for the 2017 season:

In [9]:
links_race_17 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=679', url_string = 'race-class')
c_17 = f1_gp_circuits(season = 2017)
city_17 = c_17['GP'].unique().tolist()

df_2021_data = fia_f1_data(2017, gp_city = city_17, gp_link = links_race_17)
df_2021_data.head()


Season: 2017 | Source: espn.com
Season: 2017 | Source: fia.com
Circuit: australian | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification
Circuit: chinese | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification-7
Circuit: bahrain | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification-0
Circuit: russian | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification-8
Circuit: spanish | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification-1
Circuit: monaco | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification-2
Circuit: canadian | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification-9
Circuit: azerbaijan | https://www.fia.com/events/fia-formula-one-world-championship/season-2017/race-classification-10
Circuit: aus

,CLASS,DRIVER,TEAM,LAPS,RACE_TIME,DRIVER_SNAME,GRAND_PRIX,SEASON,FLAP_POS,FLAP_TIME,F_LAP,FLAP_GAP,FLAP_KM/H,FLAP_HOUR,BS1_POS,BS1_TIME,BS2_POS,BS2_TIME,BS3_POS,BS3_TIME,ST_POS,ST_KM/H,I1_POS,I1_KM/H,I2_POS,I2_KM/H,FL_POS,FL_KM/H,DRIVER_NO,STOPS,PS_TOTAL_TIME
0,1,Sebastian Vettel,Scuderia Ferrari,57,1:24:11.672,S. Vettel,Australian,2017,3.0,1:26.638,53.0,0.100,220.351,17:24:49,6.0,29.083,2.0,23.164,1.0,34.385,9.0,314.6,4.0,280.8,3.0,298.8,5.0,305.1,5,1,21.988
1,2,Lewis Hamilton,Mercedes AMG Petronas F1 Team,57,1:24:21.647,L. Hamilton,Australian,2017,6.0,1:27.033,44.0,0.495,219.351,17:11:49,3.0,28.989,6.0,23.243,6.0,34.584,4.0,318.0,2.0,283.9,1.0,300.0,2.0,308.3,44,1,21.709
2,3,Valtteri Bottas,Mercedes AMG Petronas F1 Team,57,1:24:22.922,V. Bottas,Australian,2017,2.0,1:26.593,56.0,0.055,220.465,17:29:23,1.0,28.885,3.0,23.168,4.0,34.453,6.0,316.3,1.0,284.9,5.0,297.9,6.0,304.9,77,1,21.440
3,4,Kimi Raikkonen,Scuderia Ferrari,57,1:24:34.065,K. Raikkonen,Australian,2017,1.0,1:26.538,56.0,0.000,220.605,17:29:33,2.0,28.903,5.0,23.225,2.0,34.410,13.0,304.3,11.0,277.2,13.0,293.2,14.0,297.7,7,1,22.033
4,5,Max Verstappen,Red Bull Racing,57,1:24:40.499,M. Verstappen,Australian,2017,5.0,1:26.964,43.0,0.426,219.525,17:10:39,7.0,29.103,1.0,23.071,5.0,34.552,8.0,315.3,3.0,281.6,2.0,299.4,9.0,302.8,33,1,22.208


No examples areprovide for the fia_f1_season function, but the usage is the same as the example above and we can see the aplication bellow.

After we run both functions we just need to merge both dataset by driver and GP, and move on to another season.

# Web Scrapping

We will now aplly the functions above for seasons from 2015 to 2021 on the fia and espn websites.

## 2021

In [10]:
links_race_21 = Links_Extraction(url = 'https://www.fia.com/f1-archives', url_string = 'race-classification')

In [11]:
links_race_21

['/events/fia-formula-one-world-championship/season-2021/bahrain-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/emilia-romagna-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/portuguese-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/spanish-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/monaco-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/azerbaijan-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/french-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/styrian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/austrian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2021/british-grand-prix/race-classification',
 '/event

In [12]:
links_session_21 = Links_Extraction(url = 'https://www.fia.com/f1-archives', url_string = 'session')

In [13]:
c_21 = f1_gp_circuits(season = 2021)
city_21 = c_21['GP'].tolist()
c_21

Season: 2021 | Source: espn.com


,Date,Race,GP
0,Mar 26 - 28,Bahrain GPBahrain International Circuit,bahrain
1,Apr 16 - 18,Emilia Romagna GPAutodromo Enzo e Dino Ferrari,emilia-romagna
2,Apr 30 - May 2,Portuguese GPAutódromo Internacional Do Algarve,portuguese
3,May 7 - 9,Spanish GPCircuit de Barcelona-Catalunya,spanish
4,May 20 - 23,Monaco GPCircuit de Monaco,monaco
...,...,...,...
17,Nov 5 - 7,Mexican GPAutodromo Hermanos Rodriguez,mexican
18,Nov 12 - 14,Brazilian GPAutodromo Jose Carlos Pace,brazilian
19,Nov 19 - 21,Qatar GPLosail International Circuit,qatar
20,Dec 3 - 5,Saudi Arabian GPJeddah Street Circuit,saudi-arabia


In [ ]:
df_2021 = fia_f1_data(2021, gp_city = city_21, gp_link = links_race_21)

In [ ]:
df_2021

In [ ]:
df_2021_session = fia_f1_session(2021, gp_city = city_21, gp_link = links_session_21)

In [ ]:
df_2021_session

In [ ]:
f1_2021 = pd.merge(df_2021, df_2021_session, how='left', left_on=['DRIVER','GRAND_PRIX'], right_on = ['DRIVER','GRAND_PRIX'])

In [ ]:
f1_2021

# 2020

In [ ]:
Links_Extraction(url = 'https://www.fia.com/f1-archives?season=1059', url_string = 'race')

In [ ]:
links_race_20 = [
'/events/fia-formula-one-world-championship/season-2020/austrian-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/styrian-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/hungarian-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/british-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/formula-1-70th-anniversary-grand-prix/race',
'/events/fia-formula-one-world-championship/season-2020/spanish-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/belgian-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/italian-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/tuscan-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/russian-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/eifel-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/portuguese-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/emilia-romagna-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/turkish-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/bahrain-grand-prix/race-classification-0',
'/events/fia-formula-one-world-championship/season-2020/sakhir-grand-prix/race-classification',
'/events/fia-formula-one-world-championship/season-2020/abu-dhabi-grand-prix/race-classification-0']

In [ ]:
Links_Extraction(url = 'https://www.fia.com/f1-archives?season=1059', url_string = 'session')

In [ ]:
links_session_20 = [
 '/events/fia-formula-one-world-championship/season-2020/austrian-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/styrian-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/hungarian-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/british-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/formula-1-70th-anniversary-grand-prix/session'
,'/events/fia-formula-one-world-championship/season-2020/spanish-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/belgian-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/italian-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/tuscan-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/russian-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/eifel-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/portuguese-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/emilia-romagna-grand-prix/session'
,'/events/fia-formula-one-world-championship/season-2020/turkish-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/bahrain-grand-prix/session-classifications-0'
,'/events/fia-formula-one-world-championship/season-2020/sakhir-grand-prix/session-classifications'
,'/events/fia-formula-one-world-championship/season-2020/abu-dhabi-grand-prix/session-classifications-0']

In [ ]:
c_20 = f1_gp_circuits(season = 2020)
city_20 = c_20['GP'].tolist()
c_20['GP'][4] = '70th-anniversary'
c_20

In [ ]:
df_2020 = fia_f1_data(2020, gp_city = city_20, gp_link = links_race_20)

In [ ]:
df_2020

In [ ]:
df_2020_session = fia_f1_session(2020, gp_city = city_20, gp_link = links_session_20)

In [ ]:
df_2020_session

In [ ]:
f1_2020 = pd.merge(df_2020, df_2020_session, how='left', left_on=['DRIVER','GRAND_PRIX'], right_on = ['DRIVER','GRAND_PRIX'])

In [ ]:
f1_2020

# 2019

In [ ]:
Links_Extraction(url = 'https://www.fia.com/f1-archives?season=971', url_string = 'race-cl')

In [ ]:
links_race_19 = ['/events/fia-formula-one-world-championship/season-2019/australian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/bahrain-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/chinese-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/azerbaijan-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/spanish-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/monaco-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/canadian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/french-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/british-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/german-grand-prix/race-classification-german',
 '/events/fia-formula-one-world-championship/season-2019/hungarian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/belgian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/italian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/singapore-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/russian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/japanese-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/united-states-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/brazilian-grand-prix/race-classification',
 '/events/fia-formula-one-world-championship/season-2019/abu-dhabi-grand-prix/race-classification']

In [ ]:
Links_Extraction(url = 'https://www.fia.com/f1-archives?season=971', url_string = 'session')

In [ ]:
links_session_19 = ['/events/fia-formula-one-world-championship/season-2019/australian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/bahrain-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/chinese-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/spanish-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/monaco-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/canadian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/french-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/british-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/german-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/hungarian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/belgian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/italian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/singapore-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/russian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/japanese-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/united-states-grand-prix/session',
 '/events/fia-formula-one-world-championship/season-2019/brazilian-grand-prix/session-classifications',
 '/events/fia-formula-one-world-championship/season-2019/abu-dhabi-grand-prix/session-classifications']

In [ ]:
c_19 = f1_gp_circuits(season = 2019)
c_19 = c_19.drop(index=8)
c_19 = c_19.drop(index=17)
city_19 = c_19['GP'].unique().tolist()
c_19_2 = c_19.drop(index=3)
c_19

In [ ]:
city_19_2 = c_19_2['GP'].tolist()

In [ ]:
df_2019 = fia_f1_data(2019, gp_city = city_19, gp_link = links_race_19)

In [ ]:
#df_2019

In [ ]:
df_2019_session = fia_f1_session(2019, gp_city = city_19_2, gp_link = links_session_19)

In [ ]:
f1_2019 = pd.merge(df_2019, df_2019_session, how='left', left_on=['DRIVER','GRAND_PRIX'], right_on = ['DRIVER','GRAND_PRIX'])

In [ ]:
f1_2019

# 2018

In [ ]:
Links_Extraction(url = 'https://www.fia.com/f1-archives?season=866', url_string = 'race-class')

In [ ]:
links_race_18 = ['/events/fia-formula-one-world-championship/season-2018/race-classification',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-0',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-1',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-25',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-3',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-23',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-5',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-6',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-7',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-8',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-24',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-26',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-27',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-28',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-13',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-29',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-30',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-22',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-17',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-18',
 '/events/fia-formula-one-world-championship/season-2018/race-classification-20']

In [ ]:
Links_Extraction(url = 'https://www.fia.com/f1-archives?season=866', url_string = 'session')

In [ ]:
links_session_18 = ['/events/fia-formula-one-world-championship/season-2018/session-classifications',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-0',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-1',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-25',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-3',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-23',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-5',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-6',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-7',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-8',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-24',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-26',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-27',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-28',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-13',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-29',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-30',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-22',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-17',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-18',
 '/events/fia-formula-one-world-championship/season-2018/session-classifications-19']

In [ ]:
c_18 = f1_gp_circuits(season = 2018)
city_18 = c_18['GP'].unique().tolist()
c_18

In [ ]:
df_2018 = fia_f1_data(2018, gp_city = city_18, gp_link = links_race_18)

In [ ]:
df_2018_session = fia_f1_session(2018, gp_city = city_18, gp_link = links_session_18)

In [ ]:
f1_2018 = pd.merge(df_2018, df_2018_session, how='left', left_on=['DRIVER','GRAND_PRIX'], right_on = ['DRIVER','GRAND_PRIX'])

In [ ]:
f1_2018

# 2017

In [ ]:
links_race_17 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=679', url_string = 'race-class')

In [ ]:
links_race_17

In [ ]:
links_session_17 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=679', url_string = 'session')

In [ ]:
links_session_17

In [ ]:
c_17 = f1_gp_circuits(season = 2017)
city_17 = c_17['GP'].unique().tolist()
c_17

In [ ]:
df_2017 = fia_f1_data(2017, gp_city = city_17, gp_link = links_race_17)

In [ ]:
#df_2017

In [ ]:
df_2017_session = fia_f1_session(2017, gp_city = city_17, gp_link = links_session_17)

In [ ]:
#df_2017_session

In [ ]:
f1_2017 = pd.merge(df_2017, df_2017_session, how='left', left_on=['DRIVER','GRAND_PRIX'], right_on = ['DRIVER','GRAND_PRIX'])

In [ ]:
f1_2017

# 2016

In [ ]:
links_race_16 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=644', url_string = 'race-class')

In [ ]:
links_race_16

In [ ]:
links_session_16 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=644', url_string = 'session')

In [ ]:
links_session_16

In [ ]:
c_16 = f1_gp_circuits(season = 2016)
city_16 = c_16['GP'].unique().tolist()
c_16

In [ ]:
df_2016 = fia_f1_data(2016, gp_city = city_16, gp_link = links_race_16)

In [ ]:
#df_2016

In [ ]:
df_2016_session = fia_f1_session(2016, gp_city = city_16, gp_link = links_session_16)

In [ ]:
f1_2016 = pd.merge(df_2016, df_2016_session, how='left', left_on=['DRIVER','GRAND_PRIX'], right_on = ['DRIVER','GRAND_PRIX'])

In [ ]:
f1_2016

# 2015

In [ ]:
links_race_15 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=249', url_string = 'race-class')

In [ ]:
links_race_15 = ['/events/fia-formula-one-world-championship/season-2015/race-classification',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-0',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-1',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-2',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-3',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-4',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-5',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-6',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-7',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-9',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-10',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-11',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-12',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-13',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-14',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-15',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-16',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-17',
 '/events/fia-formula-one-world-championship/season-2015/race-classification-18']

In [ ]:
links_session_15 = Links_Extraction(url = 'https://www.fia.com/f1-archives?season=249', url_string = 'session')

In [ ]:
links_session_15 = ['/events/fia-formula-one-world-championship/season-2015/session-classifications-0',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-1',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-2',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-3',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-4',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-5',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-6',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-7',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-8',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-10',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-11',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-12',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-13',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-14',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-15',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-16',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-17',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications-18',
 '/events/fia-formula-one-world-championship/season-2015/session-classifications']

In [ ]:
c_15 = f1_gp_circuits(season = 2015)
city_15 = c_15['GP'].unique().tolist()
c_15

In [ ]:
df_2015 = fia_f1_data(2015, gp_city = city_15, gp_link = links_race_15)

In [ ]:
df_2015_session = fia_f1_session(2015, gp_city = city_15, gp_link = links_session_15)

In [ ]:
f1_2015 = pd.merge(df_2015, df_2015_session, how='left', left_on=['DRIVER','GRAND_PRIX'], right_on = ['DRIVER','GRAND_PRIX'])

In [ ]:
data = f1_2021.append(f1_2020, ignore_index = True)
data = data.append(f1_2019, ignore_index = True)
data = data.append(f1_2018, ignore_index = True)
data = data.append(f1_2017, ignore_index = True)
data = data.append(f1_2016, ignore_index = True)
data = data.append(f1_2015, ignore_index = True)

In [ ]:
print('Shape: ' + str(data.shape))
data.head()

The first part of the dataset preparation is concluded. We can export the data on to excel and take a first glance at the extracted data.

On the next chapter we will do some preprocessing on some data.

## Pre Processing

Update the pilots names to be consistent throught the dataset, since names change from season to season. Also, there are also some caracters that weren't correct on the website and we need to correct it.

These updates will allow us to merge more data from other data sources in order to complete our dataset.

In [ ]:
df = data.copy() #checkpoint

In [ ]:
df.drop(['SEASON_y'], axis = 1, inplace=True)
df.rename(columns= {'SEASON_x':'SEASON'}, inplace=True)

In [ ]:
df['DRIVER'] = df['DRIVER'].str.replace('Sergio Perez Mendoza','Sergio Perez')
df['DRIVER_SNAME'] = df['DRIVER_SNAME'].str.replace('S. Perez Mendoza','S. Perez')

In [ ]:
df['DRIVER'] = df['DRIVER'].str.replace(r'[^\x00-\x7f]', '')
df['DRIVER'] = df['DRIVER'].str.replace('Esteban Gutirrez','Esteban Gutierrez')
df['DRIVER_SNAME'] = df['DRIVER_SNAME'].str.replace(r'[^\x00-\x7f]', '')
df['DRIVER_SNAME'] = df['DRIVER_SNAME'].str.replace('E. Gutirrez','E. Gutierrez')

In [ ]:
df['DRIVER'] = df['DRIVER'].str.replace('Carlos Sainz Jr.','Carlos Sainz')
df['DRIVER_SNAME'] = df['DRIVER_SNAME'].str.replace('C. Sainz Jr.','C. Sainz')

In [ ]:
df['DRIVER'] = df['DRIVER'].str.replace('Roberto Merhi Muntan','Roberto Merhi')
df['DRIVER_SNAME'] = df['DRIVER_SNAME'].str.replace('R. Merhi Muntan','R. Merhi')

In [ ]:
df['GRAND_PRIX'] = df['GRAND_PRIX'].str.lower()

In [ ]:
web_data = df.copy() #checkpoint 2

In [ ]:
web_data

## Azure Database

We would also like to demonstrate the connection from python notebooks to an Azure SQL database to retrive some information to add to our dataset.

On the following site http://ergast.com/mrd/db/#csv we have downloaded the MySQL (5.7) data dumps and convert it to T-SQL in order to run this on Azure SQL Server database.

After that we just need to setup the connection by getting the server, database, username and password.

In [ ]:
server = 'f1server.database.windows.net'
database = 'WackyRacesF1'
username = 'PDS'
password = '{Formula1}'  

We now needd to establish a connection with the database.

In [ ]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+ database +';UID='+username+';PWD='+ password)

Now, let's test the connection by doing a simple query on any table, for exampl select the top 5 rows from the table circuits. We setup our variable query and the results from the query to the variable df and then we just need to call it.

In [ ]:
query = 'select top 5 * from dbo.circuits'

df = pd.read_sql_query(query, cnxn)
df

<div class="alert alert-block alert-info">
<b>Note:</b> read_sql_query will allow the information returned to be displayed as an actual table, that we are used to use in pandas.
</div>

### Retrieve Information from Azure SQL Server

We want to retrive the list of all drivers with the corresponding **date of birth** and **nationality** we might have on our scrapped dataset.

In [ ]:
query = '''
    SELECT [driverId] AS DRIVER_ID
	    , [forename] + ' '+ [surname] AS DRIVER
        , [driverRef] as DRIVER_REF
	    , [code] AS DRIVER_CODE
        , [dob] AS DOB
        , [nationality] AS NATIONALITY
    FROM [dbo].[drivers]
    WHERE [number] IS NOT NULL
    '''
driver = pd.read_sql_query(query, cnxn)

In [ ]:
driver.head()

Additionally, we have retrived DRIVERREF, and DRIVER_CODE, which may come in handy to merge or display information in some graphics in the exploration phase.

Before we can use this data we nee to perform some preprocessing to the names of the drivers to make them consistent among all datsets.

In [ ]:
driver['DRIVER'] = driver['DRIVER'].str.replace('Kimi Räikkönen','Kimi Raikkonen')
driver['DRIVER'] = driver['DRIVER'].str.replace('Nico Hülkenberg','Nico Hulkenberg')
driver['DRIVER'] = driver['DRIVER'].str.replace('Sergio Pérez','Sergio Perez')
driver['DRIVER'] = driver['DRIVER'].str.replace('Jean-Éric Vergne','Jean-Eric Vergne')
driver['DRIVER'] = driver['DRIVER'].str.replace('Esteban Gutiérrez','Esteban Gutierrez')

In [ ]:
driver.head()

We have processed the data to use at a later date. We can now store this data in the database under a new schema.

we will create a cursor to create a new schema in the database

In [ ]:
#cursor = cnxn.cursor()

In [ ]:
# This was already run and a schema was created to store our data

#cursor.execute('CREATE SCHEMA pfds')
#cnxn.commit()


Before uploading the data into the database we need to create the table that will hold our information.

In [ ]:
# No need to recreate the table, we will just truncate the table.

#cursor.execute('''
#	CREATE TABLE pfds.driver (
#	DRIVER_ID int,
#	DRIVER nvarchar(50),
#	DRIVER_REF nvarchar(50),
#	DRIVER_CODE nvarchar(3),
#	NATIONALITY nvarchar(50)
#	)
#    ''')
#cnxn.commit()

In [ ]:
# will remove all data from pfds.driver table in the Azure SQL server

#cursor.execute('TRUNCATE TABLE pfds.driver')
#cnxn.commit()

In [ ]:
# Inserts the results from trhe query back to the Azure SQL Server database

#for row in driver.itertuples():
#    cursor.execute('''
#                INSERT INTO pfds.driver (DRIVER_ID, DRIVER, DRIVER_REF, DRIVER_CODE, NATIONALITY)
#                VALUES (?,?,?,?,?)
#                ''',
#                row.DRIVER_ID, 
#                row.DRIVER,
#                row.DRIVER_REF,
#                row.DRIVER_CODE,
#                row.NATIONALITY
#                )
#cnxn.commit()

Note that this code was commented out, since this will drop and recreate tables in the server and this does take some time. Please feel free to do so in order to test these features.

After we have imported the data into Azure SQL server we can query that data by just:

In [ ]:
query = 'SELECT * FROM pfds.driver'

azure_driver = pd.read_sql_query(query, cnxn)

In [ ]:
azure_driver

Let us save our data into an excel file.

In [ ]:
driver.to_excel('f1_drivers_2015_2021.xlsx', index = False)

Now we need to retrieve information regarding the status of the race for each racer, meaning, we need to know if a driver finished the race or if had a malfunction and if so, which one was.

Let us do another query, retrieving information from several tables including the one we have just uploaded.

In [ ]:
query = ''';WITH one AS (SELECT 
	  drivers.driverId AS DRIVER_ID
	, D.DRIVER
	, results.number AS NUMBER
	, results.raceId AS RACE_ID
	, races.name as RACE_NAME
	, replace(races.name,' Grand Prix', '') AS RACE 
	, results.StatusId AS STATUS_ID
	, status.status AS DRIVER_STATUS
	, circuits.circuitId AS CIRCUIT_ID
	, circuits.circuitREf AS CIRCUIT_REF
	, races.year AS SEASON
	FROM drivers
	left join results
		ON drivers.driverId = results.driverId
	left join races
		ON results.raceId = races.raceId
	left join seasons
		ON races.year = seasons.year
	left join circuits
		ON  races.circuitId = circuits.circuitId
	left join status
		ON  results.statusId = status.statusId
	left join pfds.driver D
		ON  drivers.driverId  = D.driver_Id)
	, two as (
	SELECT distinct * , replace(lower(RACE),' ', '-') AS GRAND_PRIX
	FROM one 
	WHERE SEASON IN ('2015','2016','2017','2018','2019','2020','2021'))
	SELECT * FROM two
	'''

status = pd.read_sql_query(query, cnxn)

When looking at the data we need to correct some information from the data base, so we run the following code

In [ ]:
status['RACE_NAME'] = status['RACE_NAME'].str.replace(r'[^\x00-\x7f]', '')
status['RACE_NAME'] = status['RACE_NAME'].str.replace('So Paulo Grand Prix','Sao Paulo Grand Prix')

status['RACE'] = status['RACE'].str.replace(r'[^\x00-\x7f]', '')
status['RACE'] = status['RACE'].str.replace('So Paulo','So Paulo')

status['GRAND_PRIX'] = status['GRAND_PRIX'].str.replace(r'[^\x00-\x7f]', '')
status['GRAND_PRIX'] = status['GRAND_PRIX'].str.replace('So Paulo','brazilian')

status['GRAND_PRIX'] = status['GRAND_PRIX'].str.replace('mexico-city','mexican')

The data should be now ready to be merged with our main data set.

In [ ]:
status

In [ ]:
# No need to recreate the table, we will just truncate the table.

#cursor.execute('''
#	CREATE TABLE pfds.status (
#      DRIVER_ID	int
#    , DRIVER	nvarchar(50)
#    , NUMBER	int
#    , RACE_ID	int
#    , RACE_NAME	nvarchar(100)
#    , RACE	nvarchar(50)
#    , STATUS_ID	int
#    , DRIVER_STATUS	nvarchar(50)
#    , CIRCUIT_ID	int
#    , CIRCUIT_REF	nvarchar(50)
#    , SEASON	nvarchar(4)
#    , GRAND_PRIX	nvarchar(50)
#	)
#    ''')
#cnxn.commit()

In [ ]:
#cursor.execute('TRUNCATE TABLE pfds.status')
#cnxn.commit()

In [ ]:
#for row in status.itertuples():
#    cursor.execute('''
#                INSERT INTO pfds.status (
#                                          DRIVER_ID	
#                                        , DRIVER
#                                        , NUMBER
#                                        , RACE_ID
#                                        , RACE_NAME	
#                                        , RACE
#                                        , STATUS_ID	
#                                        , DRIVER_STATUS
#                                        , CIRCUIT_ID	
#                                        , CIRCUIT_REF	
#                                        , SEASON
#                                        , GRAND_PRIX	
#                                        )
#                VALUES (?,?,?,?,?,?,?,?,?,?,?,?)
#                ''',
#                row.DRIVER_ID, 
#                row.DRIVER,
#                row.NUMBER,
#                row.RACE_ID,
#                row.RACE_NAME,
#                row.RACE,
#                row.STATUS_ID,	
#                row.DRIVER_STATUS,
#                row.CIRCUIT_ID,	
#                row.CIRCUIT_REF,
#                row.SEASON,
#                row.GRAND_PRIX,	
#                )
#cnxn.commit()

In [ ]:
query = 'SELECT * FROM pfds.status'

azure_status = pd.read_sql_query(query, cnxn)

In [ ]:
azure_status

In [ ]:
f1_full_data = pd.merge(web_data, status, how='left', left_on=['DRIVER','GRAND_PRIX','SEASON'], right_on = ['DRIVER','GRAND_PRIX','SEASON'])

In [ ]:
f1_full_data.head()

In [ ]:
f1_full_data.info()

Let us save our data set in excel format.

In [ ]:
f1_full_data.to_excel('f1_race_data_2015_2021.xlsx')

# API

We can also make use of API functionality that will let us get the most updated information if we would require the dat to be updated constantely, which is not the case but we will make a demonstration of it.

We will use the API to request all the circuits in use for seasons 2015 to 2021, this will retrieve all the circuits locations that we require for our work.

In [ ]:
# query API

races = {'season': [],
        'circuit_id': [],
        'lat': [],
        'long': [],
        'country': []}

for year in list(range(2015,2021)):
    
    url = 'https://ergast.com/api/f1/{}.json' 
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        try:
            races['season'].append(int(item['season']))
        except:
            races['season'].append(None)

        try:
            races['circuit_id'].append(item['Circuit']['circuitId'])
        except:
            races['circuit_id'].append(None)

        try:
            races['lat'].append(float(item['Circuit']['Location']['lat']))
        except:
            races['lat'].append(None)

        try:
            races['long'].append(float(item['Circuit']['Location']['long']))
        except:
            races['long'].append(None)

        try:
            races['country'].append(item['Circuit']['Location']['country'])
        except:
            races['country'].append(None)
        
races = pd.DataFrame(races)

Let us check the results from our query.

In [ ]:
races

In [ ]:
f1_circuits = races
f1_circuits.to_excel('f1_circuits_2015_2021.xlsx', index= False) 

# Data extracted 

Our main data is comprised of 3 datasets

1. Drivers
2. Circuits
3. Race data

### 1. Drivers
- **DRIVER_ID**:	    Driver Identification
- **DRIVER**:	        Driver name
- **DRIVER_REF**:	    Driver reference (last name)
- **DRIVER_CODE**:	    3 letter code based on the driver name
- **DOB**:	            Date of birth
- **NATIONALITY**:	    Nationality
    
### 2. Circuits 
- **season**:           Season
- **circuit_id**:       Circuit identification
- **lat**:              Latitude
- **long**:             Longitude
- **country**:          Country

### 3. Race data

- **CLASS**:	        Race position
- **DRIVER**:	        Driver name
- **TEAM**:	            F1 Team
- **LAPS**:	            Laps completed
- **RACE_TIME**:	    Race total time
- **DRIVER_SNAME**:	    Driver short name
- **GRAND_PRIX**:	    Grand Prix 
- **SEASON**:	        Season year
- **FLAP_POS**:	        Fastest lap position
- **FLAP_TIME**:	    Fastest lap time
- **F_LAP**:	        Fastest lap
- **FLAP_GAP**:	        Fastest lap gap
- **FLAP_KM/H**:	    Fastest lap max speed
- **FLAP_HOUR**:	    Fastest lap time of the day
- **BS1_POS**:	        Sector 1 position
- **BS1_TIME**:	        Sector 1 time
- **BS2_POS**:	        Sector 2 position
- **BS2_TIME**:	        Sector 2 time
- **BS3_POS**:	        Sector 3 position
- **BS3_TIME**:	        Sector 3 time
- **ST_POS**:	        Speed trap position
- **STKM/H**:	        Speed trap speed
- **I1_POS**:	        Maximum speed intermediate 1 position
- **I1_KM/H**:	        Maximum speed intermediate 1 speed
- **I2_POS**:	        Maximum speed intermediate 2 position
- **I2_KM/H**:	        Maximum speed intermediate 2 speed
- **FL_POS**:	        Maximum speed finish line position
- **FL_KM/H**:	        Maximum speed finish line speed
- **DRIVER_NO**:	    Driver number
- **STOPS**:	        Number of pit stops
- **PS_TOTAL_TIME**:	Pit stops total time
- **QL_CLASS**:	        Session classification position
- **QL_TIME1**:	        Qualification session 1 time
- **QL_LAPS1**:	        Qualification session 1 laps
- **QL_TIME2**:	        Qualification session 2 time
- **QL_LAPS2**:	        Qualification session 2 laps
- **QL_TIME3**:	        Qualification session 3 time
- **QL_LAPS3**:	        Qualification session 3 laps
- **GD_CLASS**:	        Grid position
- **GD_TIME**:	        Grid time
- **DRIVER_ID**:	    Driver identification
- **DRIVER**:	        Driver name
- **NUMBER**:	        Driver number
- **RACE_ID**:          Race identification
- **RACE_NAME**:	    Race name
- **RACE**:	            Race short name
- **STATUS_ID**:	    Driver status for each race
- **DRIVER_STATUS**:	Decode for Status ID
- **CIRCUIT_ID**:	    Circuit identification
- **CIRCUIT_REF**:	    Circuit reference
